In [ ]:
import requests, json

azure_client_id = "<client_id>"
azure_tenant_id = "<tenant_id>"
azure_client_secret = "<client_secret>"

synapse_workspace_name = "<synapse_workspace_name>"
output_folder = "<fabric_lakehouse_abfss_path_to_files_section>"

In [ ]:
def get_access_token():
    url = f"https://login.microsoftonline.com/{azure_tenant_id}/oauth2/token"

    payload = {
        "grant_type": "client_credentials",
        "client_id": {azure_client_id},
        "client_secret": {azure_client_secret},
        "resource": f"https://dev.azuresynapse.net/"
    }

    headers = {'Content-Type': 'application/x-www-form-urlencoded'}

    response = requests.post(url, data=payload, headers=headers)
    response_json = json.loads(response.text)
    synapse_dev_token = response_json["access_token"]

    return synapse_dev_token

In [ ]:
def clean_notebook_cells(ntbk_json, tags_to_clean):
    for cell in ntbk_json['cells']:
        for tag in tags_to_clean:
             if tag in cell:
                cell[tag] = []

    return ntbk_json

def export_notebooks():

    base_uri = f"{synapse_workspace_name}.dev.azuresynapse.net"
    resource_type = "notebooks"
    api_version = "2020-12-01"
    synapse_dev_token = get_access_token()
    notebooks_exported = 0

    url = f"https://{base_uri}/{resource_type}?api-version={api_version}"

    headers = {
        'Authorization': f'Bearer {synapse_dev_token}',
        'Content-Type': 'application/json'
    }

    response = requests.request("GET", url, headers=headers)
    resources_exported = {}

    if response != None and (response.status_code == 202 or response.status_code == 200):
        response_json = response.json()
        print(f"Getting Individual Resources of type '{resource_type}' ...")
        if "value" in response_json:
            response_json = response_json['value']
        elif "items" in response_json:
            response_json = response_json['items']
        for artifact in response_json:
            if "name" in artifact:
                resource_name = artifact["name"]
            elif "Name" in artifact:
                resource_name = artifact["Name"]
            print(f"Getting Resource '{resource_name}' ...")
            resource_url = f"https://{base_uri}/{resource_type}/{resource_name}?api-version={api_version}"
            resource_response = requests.request("GET", resource_url, headers=headers)

            if resource_response != None and (resource_response.status_code == 202 or resource_response.status_code == 200):
                resource_response_json = resource_response.json()

                if (resource_type == "notebooks"):
                    notebook_json = resource_response_json['properties']
                    tags_to_clean = ['outputs']
                    updated_ntbk_json = clean_notebook_cells(notebook_json, tags_to_clean)
                    file_name = f"{resource_name}.ipynb"

                    data = json.dumps(updated_ntbk_json, indent=4)
                    mssparkutils.fs.put(f"{output_folder}/export/{synapse_workspace_name}/{resource_type}/{file_name}", data, False)
                    notebooks_exported += 1
                    resources_exported[resource_type] = notebooks_exported
                
    else:
        raise RuntimeError(f"Getting items of type '{resource_type}' failed: {response.status_code}: {response}")

    print(f"Finish exporting {resources_exported[resource_type]} items of type: {resource_type}")

In [ ]:
export_notebooks()